In [1]:
from bs4 import BeautifulSoup
import thulac
import pickle

with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

with open('max_words.pickle', 'rb') as handle:
    load_max_words = pickle.load(handle)

with open('categories.pickle', 'rb') as handle:
    categories = pickle.load(handle)

Using TensorFlow backend.


In [2]:
thu = thulac.thulac(seg_only=True)
def clean_and_cut(text):
    soup = BeautifulSoup(text, 'html.parser')
    new_text = soup.get_text()
    
    new_text_cut = thu.cut(new_text, text=True).replace("\n", "")
    return new_text_cut

Model loaded succeed


In [3]:
example_text = """
再發霜凍警告21人低溫入院兩人危殆

寒冷天氣持續，天文台在下午再度發出霜凍警告，指明早新界北部可能出現地面霜。寒冷天氣警告及紅色火災危險警告繼續生效。天文台預測，明早市區最低氣溫約8度，新界再低兩三度，日間最高氣溫約13度。有菜農指，若果結霜，只能提早收割。

醫管局指，在截至下午五時的過去二十四小時，公立醫院合共收到二十一名病人因低溫而需送往急症室治療，包括九男十二女，年齡介乎五十三至九十三歲，當中兩人危殆、六人嚴重。
"""

In [4]:
from keras.preprocessing.sequence import pad_sequences

new_text = clean_and_cut(example_text)
print(new_text)

new_X = load_tokenizer.texts_to_matrix(new_text)

new_X = pad_sequences(new_X, maxlen=load_max_words, padding='post')
print(new_X)

再 發 霜 凍 警告 21 人 低 溫 入院 兩 人 危殆寒冷 天氣 持續 ， 天文台 在 下午 再度 發出霜 凍 警告 ， 指明 早 新界 北部 可能 出現 地面霜 。 寒冷 天氣 警告 及 紅色 火災 危險 警告 繼續 生效 。 天文台 預測 ， 明早市 區 最低 氣溫約 8 度 ， 新界 再 低 兩 三 度 ， 日間 最高 氣溫約 13 度 。 有 菜 農指 ， 若 果結霜 ， 只能 提 早 收割 。醫管局 指 ， 在 截至 下午 五時 的 過 去 二十四 小 時 ， 公立 醫院 合共 收到 二十一 名 病人 因 低溫 而 需 送 往 急症室 治療 ， 包括 九 男 十二 女 ， 年齡 介乎 五十三 至 九十三 歲 ， 當 中 兩 人 危殆 、 六 人 嚴重 。
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [6]:
import numpy as np
from keras.metrics import top_k_categorical_accuracy
from keras.models import load_model

def top_1_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=1)

model = load_model('text_full_model.h5', custom_objects={'top_1_accuracy': top_1_accuracy})

def top_1_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=1)

predict_class = model.predict_classes(new_X)

338/338 [==============================] - 1s     


In [7]:
from collections import Counter
cls_counter = Counter(predict_class)

idx = cls_counter.most_common(1)[0][0]

final_category = np.sort(list(categories))[2]

print(final_category)

C
